# Simplified Lora Trainer by Citron Legacy 🍋

Help fuel my passion!
 [![ko-fi](https://img.shields.io/badge/Support%20me%20on%20Ko--fi-F16061?logo=ko-fi&logoColor=white&style=flat)](https://ko-fi.com/citronlegacy)

Any amount would be awesome!



### Project Description

This Google Colab notebook is a project for simplying the training of Loras for Stable Diffusion. There are a lot of great Lora training tools with nice features but this one is intend to hide advanced settings and make the simplest trainer possible.




---
### Project Disclaimer
This is forked from the work of [Hollowstrawberry 🍓](https://github.com/hollowstrawberry/kohya-colab) which is based on the work of [Kohya-ss](https://github.com/kohya-ss/sd-scripts) and [Linaqruf](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-dreambooth.ipynb). Thank you!

The purpose of this document is to research bleeding-edge technologies in the field of machine learning inference.  
Please read and follow the [Google Colab guidelines](https://research.google.com/colaboratory/faq.html) and its [Terms of Service](https://research.google.com/colaboratory/tos_v3.html).

---

# Links
| |GitHub| Colab | |
|:--|:-:|:-:|:-:|
| 🏠 **Homepage** | [![GitHub](https://raw.githubusercontent.com/citronlegacy/kohya-colab/main/assets/github.svg)](https://github.com/citronlegacy/kohya-colab) | | |
| 📊 **Citron Dataset Maker** | [![GitHub](https://raw.githubusercontent.com/citronlegacy/kohya-colab/main/assets/github.svg)](https://github.com/citronlegacy/kohya-colab/blob/main/Citron_Dataset_Maker.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/citronlegacy/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/citronlegacy/kohya-colab/blob/main/Citron_Dataset_Maker.ipynb) |
| 💪 **Citron Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/citronlegacy/kohya-colab/main/assets/github.svg)](https://github.com/citronlegacy/kohya-colab/blob/main/Citron_Lora_Trainer.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/citronlegacy/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/citronlegacy/kohya-colab/blob/main/Citron_Lora_Trainer.ipynb) |


---

In [ ]:
#@title # Install
#@markdown ### 1️⃣ Setup Connect to Google Drive and Install Dependences
#@markdown Installation usually takes about 3 minutes

#@markdown ------------------------------------------------------

#@markdown Update 02/02/2024 - First release of my Citron Dataset Maker!

import os
import time
import re
import toml
import shutil
import zipfile
from IPython.display import Markdown, display
from datetime import datetime, timedelta
from pathlib import Path
from google.colab.output import clear as clear_output
from google.colab import drive


global dependencies_installed
dependencies_installed = False

print("📂 Connecting to Google Drive...")
drive.mount('/content/drive')

root_dir = "/content"
deps_dir = os.path.join(root_dir, "deps")
repo_dir = os.path.join(root_dir, "kohya-trainer")
COLAB = True # low ram
COMMIT = "e6ad3cbc66130fdc3bf9ecd1e0272969b1d613f7"
BETTER_EPOCH_NAMES = True
LOAD_TRUNCATED_IMAGES = True

def clone_repo():
  os.chdir(root_dir)
  !git clone https://github.com/kohya-ss/sd-scripts {repo_dir}
  os.chdir(repo_dir)
  if COMMIT:
    !git reset --hard {COMMIT}
  !wget https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/requirements.txt -q -O requirements.txt

def install_dependencies():
  clone_repo()
  !apt -y update -qq
  !apt -y install aria2 -qq
  !pip install --upgrade -r requirements.txt

  # patch kohya for minor stuff
  if COLAB:
    !sed -i "s@cpu@cuda@" library/model_util.py # low ram
  if LOAD_TRUNCATED_IMAGES:
    !sed -i 's/from PIL import Image/from PIL import Image, ImageFile\nImageFile.LOAD_TRUNCATED_IMAGES=True/g' library/train_util.py # fix truncated jpegs error
  if BETTER_EPOCH_NAMES:
    !sed -i 's/{:06d}/{:02d}/g' library/train_util.py # make epoch names shorter
    !sed -i 's/"." + args.save_model_as)/"-{:02d}.".format(num_train_epochs) + args.save_model_as)/g' train_network.py # name of the last epoch will match the rest

  from accelerate.utils import write_basic_config
  accelerate_config_file = os.path.join(repo_dir, "accelerate_config/config.yaml")
  if not os.path.exists(accelerate_config_file):
    write_basic_config(save_location=accelerate_config_file)

  os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
  os.environ["BITSANDBYTES_NOWELCOME"] = "1"
  os.environ["SAFETENSORS_FAST_GPU"] = "1"

  global dependencies_installed
  dependencies_installed = True

aiInstalls_start_time = time.perf_counter()
install_dependencies()
aiInstalls_end_time = time.perf_counter()


def count_images_in_folder(folder_path):
    # Ensure the folder path is a valid directory
    folder_path = Path(folder_path)
    if not folder_path.is_dir():
        raise ValueError(f"The provided path '{folder_path}' is not a valid directory.")

    # Count the number of image files in the folder
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp']
    image_count = sum(1 for file in folder_path.iterdir() if file.suffix.lower() in image_extensions)

    return image_count

#Import colabUtilities
!git clone https://github.com/citronlegacy/kohya-colab.git
# CD into project
%cd kohya-colab
# import modules
import colabUtilities
#return to original directory
%cd ..

print("AI Installation time: " + str(colabUtilities.get_time_hh_mm_ss(aiInstalls_end_time-aiInstalls_start_time)) + " minutes")

In [ ]:
#@title Lora Trainer

#@markdown ###Define your dataset location
#@markdown Your project name will be the same as the folder containing your images. Spaces aren't allowed.
project_name = "Hatsune_Miku" #@param {type:"string"}
#@markdown Is your dataset in Google Drive or Gogole Colab? (This hasn't been fully tested with datasets in Colab)
environment_location = "Google Drive (/content/drive/MyDrive/)" #@param ["Google Drive (/content/drive/MyDrive/)", "Google Colab (/content/)"]


folder_structure = "Organize by category (MyDrive/lora_training/datasets/project_name)"

#@markdown Decide the model that will be downloaded and used for training. You can also choose your own by pasting its download link.

#@markdown 💡 Tip from [Citron Legacy](https://civitai.com/user/CitronLegacy/models): Use AnyLora if you want to train an art style.
training_model = "Anime (animefull-final-pruned-fp16.safetensors)" #@param ["Anime (animefull-final-pruned-fp16.safetensors)", "AnyLora (AnyLoRA_noVae_fp16-pruned.ckpt)", "Stable Diffusion (sd-v1-5-pruned-noema-fp16.safetensors)"]
optional_custom_training_model_url = "" #@param {type:"string"}
custom_model_is_based_on_sd2 = False #@param {type:"boolean"}

if "Drive" in environment_location:
    environment_path = "/content/drive/MyDrive/"

elif "Colab" in environment_location:
    environment_path = ""

main_dir      = os.path.join(environment_path, "lora_training")
images_folder = os.path.join(main_dir, "datasets", project_name)
output_folder = os.path.join(main_dir, "output", project_name)
config_folder = os.path.join(main_dir, "config", project_name)
log_folder    = os.path.join(main_dir, "log")

assert count_images_in_folder(images_folder) > 0, f"Error: No images found in the specified folder: {images_folder}."

global total_steps
total_steps = ""

if optional_custom_training_model_url:
  model_url = optional_custom_training_model_url
elif "AnyLora" in training_model:
  model_url = "https://huggingface.co/Lykon/AnyLoRA/resolve/main/AnyLoRA_noVae_fp16-pruned.ckpt"
elif "Anime" in training_model:
  model_url = "https://huggingface.co/hollowstrawberry/stable-diffusion-guide/resolve/main/models/animefull-final-pruned-fp16.safetensors"
else:
  model_url = "https://huggingface.co/hollowstrawberry/stable-diffusion-guide/resolve/main/models/sd-v1-5-pruned-noema-fp16.safetensors"

if "AnyLora" in training_model:
  training_modelLogName = "AnyLoRA_noVae_fp16-pruned"
elif "Anime" in training_model:
  training_modelLogName = "Animefull-final-pruned-fp16"
elif "Stable Diffusion" in training_model:
  training_modelLogName = "sd-v1-5-pruned-noema-fp16"
else:
  training_modelLogName = "other"

# These carry information from past executions
if "model_url" in globals():
  old_model_url = model_url
else:
  old_model_url = None
if "dependencies_installed" not in globals():
  dependencies_installed = False
if "model_file" not in globals():
  model_file = None

# These may be set by other cells, some are legacy
if "custom_dataset" not in globals():
  custom_dataset = None
if "override_dataset_config_file" not in globals():
  override_dataset_config_file = None
if "override_config_file" not in globals():
  override_config_file = None
if "optimizer" not in globals():
  optimizer = "AdamW8bit"
if "optimizer_args" not in globals():
  optimizer_args = None
if "continue_from_lora" not in globals():
  continue_from_lora = ""
if "weighted_captions" not in globals():
  weighted_captions = False
if "adjust_tags" not in globals():
  adjust_tags = False
if "keep_tokens_weight" not in globals():
  keep_tokens_weight = 1.0

COLAB = True # low ram
XFORMERS = True
COMMIT = "9a67e0df390033a89f17e70df5131393692c2a55"
BETTER_EPOCH_NAMES = True
LOAD_TRUNCATED_IMAGES = True

#@markdown ### ▶️ Processing
#@markdown Resolution of 512 is standard for Stable Diffusion 1.5. Higher resolution training is much slower but can lead to better details. <p>
#@markdown Images will be automatically scaled while training to produce the best results, so you don't need to crop or resize anything yourself.
resolution = 512 #@param {type:"slider", min:512, max:1024, step:128}
#@markdown This option will train your images both normally and flipped, for no extra cost, to learn more from them. Turn it on specially if you have less than 20 images. <p>
#@markdown **Turn it off if you care about asymmetrical elements in your Lora**.
flip_aug = False #@param {type:"boolean"}
caption_extension = ".txt"
shuffle_tags = True
shuffle_caption = shuffle_tags
activation_tags = "1"
keep_tokens = int(activation_tags)

#@markdown ### ▶️ Steps - Your images will repeat this number of times during training.

#@markdown 💡 Tip from [Hollowstrawberry](https://github.com/hollowstrawberry/kohya-colab): I recommend that your images multiplied by their repeats is between 200 and 400.

#@markdown 💡 Tip from [Citron Legacy](https://civitai.com/user/CitronLegacy/models): Use 10 Repeats (or less if you have too many steps and training takes longer than 2 hours.)


num_repeats = 10 #@param {type:"number"}
#@markdown Choose how long you want to train for. A good starting point is around 10 epochs or around 2000 steps.<p>
#@markdown One epoch is a number of steps equal to: your number of images multiplied by their repeats, divided by batch size. <p>

#@markdown 💡 Tip from [Citron Legacy](https://civitai.com/user/CitronLegacy/models): Always use 10 Epochs until you know what you are doing.

preferred_unit = "Epochs" #@param ["Epochs", "Steps"]
how_many = 10 #@param {type:"number"}
max_train_epochs = how_many if preferred_unit == "Epochs" else None
max_train_steps = how_many if preferred_unit == "Steps" else None
#@markdown Saving more epochs will let you compare your Lora's progress better.
save_every_n_epochs = 5 #@param {type:"number"}
keep_only_last_n_epochs = 10 #@param {type:"number"}
if not save_every_n_epochs:
  save_every_n_epochs = max_train_epochs
if not keep_only_last_n_epochs:
  keep_only_last_n_epochs = max_train_epochs
#@markdown Increasing the batch size makes training faster, but may make learning worse. Recommended 2 or 3.
train_batch_size = 2 #@param {type:"slider", min:1, max:8, step:1}

unet_lr = 5e-4
text_encoder_lr = 1e-4
lr_scheduler = "cosine_with_restarts"
lr_scheduler_number = 3
lr_scheduler_num_cycles = lr_scheduler_number if lr_scheduler == "cosine_with_restarts" else 0
lr_scheduler_power = lr_scheduler_number if lr_scheduler == "polynomial" else 0
lr_warmup_ratio = 0.05
lr_warmup_steps = 0
min_snr_gamma = True
min_snr_gamma_value = 5.0 if min_snr_gamma else None

#@markdown Dim - More dim means larger Lora, it can hold more information but more isn't always better.
lora_type = "LoRA"
dim_to_use = "Dim=32; Alpha=16" #@param ["Dim=16; Alpha=8", "Dim=32; Alpha=16", "Dim=64; Alpha=32"]
dim_to_use_text = ""


network_dim = 32
network_alpha = 16

if (dim_to_use == "Dim=16; Alpha=8"):
  network_dim = 16
  network_alpha = 8
  dim_to_use_text = "Dim 16 and Alpha 8"
elif (dim_to_use == "Dim=32; Alpha=16"):
  network_dim = 32
  network_alpha = 16
  dim_to_use_text = "Dim 32 and Alpha 16"
elif (dim_to_use == "Dim=64; Alpha=32"):
  network_dim = 64
  network_alpha = 32
  dim_to_use_text = "Dim 64 and Alpha 32"

conv_dim = 8
conv_alpha = 4

network_module = "networks.lora"
network_args = None
if lora_type.lower() == "locon":
  network_args = [f"conv_dim={conv_dim}", f"conv_alpha={conv_alpha}"]



# 👩‍💻 Cool code goes here

if optimizer.lower() == "prodigy" or "dadapt" in optimizer.lower():
  if override_values_for_dadapt_and_prodigy:
    unet_lr = 0.5
    text_encoder_lr = 0.5
    lr_scheduler = "constant_with_warmup"
    lr_warmup_ratio = 0.05
    network_alpha = network_dim

  if not optimizer_args:
    optimizer_args = ["decouple=True","weight_decay=0.01","betas=[0.9,0.999]"]
    if optimizer == "Prodigy":
      optimizer_args.extend(["d_coef=2","use_bias_correction=True"])
      if lr_warmup_ratio > 0:
        optimizer_args.append("safeguard_warmup=True")
      else:
        optimizer_args.append("safeguard_warmup=False")

root_dir = "/content"
deps_dir = os.path.join(root_dir, "deps")
repo_dir = os.path.join(root_dir, "kohya-trainer")




config_file = os.path.join(config_folder, "training_config.toml")
dataset_config_file = os.path.join(config_folder, "dataset_config.toml")
accelerate_config_file = os.path.join(repo_dir, "accelerate_config/config.yaml")


def validate_dataset():
  global lr_warmup_steps, lr_warmup_ratio, caption_extension, keep_tokens, keep_tokens_weight, weighted_captions, adjust_tags
  supported_types = (".png", ".jpg", ".jpeg", ".webp", ".bmp")

  print("\n💿 Checking dataset...")
  if not project_name.strip() or any(c in project_name for c in " .()\"'\\/"):
    print("💥 Error: Please choose a valid project name.")
    return

  if custom_dataset:
    try:
      datconf = toml.loads(custom_dataset)
      datasets = [d for d in datconf["datasets"][0]["subsets"]]
    except:
      print(f"💥 Error: Your custom dataset is invalid or contains an error! Please check the original template.")
      return
    reg = [d.get("image_dir") for d in datasets if d.get("is_reg", False)]
    datasets_dict = {d["image_dir"]: d["num_repeats"] for d in datasets}
    folders = datasets_dict.keys()
    files = [f for folder in folders for f in os.listdir(folder)]
    images_repeats = {folder: (len([f for f in os.listdir(folder) if f.lower().endswith(supported_types)]), datasets_dict[folder]) for folder in folders}
  else:
    reg = []
    folders = [images_folder]
    files = os.listdir(images_folder)
    images_repeats = {images_folder: (len([f for f in files if f.lower().endswith(supported_types)]), num_repeats)}

  for folder in folders:
    if not os.path.exists(folder):
      print(f"💥 Error: The folder {folder.replace('/content/drive/', '')} doesn't exist.")
      return
  for folder, (img, rep) in images_repeats.items():
    if not img:
      print(f"💥 Error: Your {folder.replace('/content/drive/', '')} folder is empty.")
      return
  for f in files:
    if not f.lower().endswith(".txt") and not f.lower().endswith(supported_types):
      print(f"💥 Error: Invalid file in dataset: \"{f}\". Aborting.")
      return

  if not [txt for txt in files if txt.lower().endswith(".txt")]:
    caption_extension = ""
  if continue_from_lora and not (continue_from_lora.endswith(".safetensors") and os.path.exists(continue_from_lora)):
    print(f"💥 Error: Invalid path to existing Lora. Example: /content/drive/MyDrive/Loras/example.safetensors")
    return

  pre_steps_per_epoch = sum(img*rep for (img, rep) in images_repeats.values())
  steps_per_epoch = pre_steps_per_epoch/train_batch_size
  global total_steps
  total_steps = max_train_steps or int(max_train_epochs*steps_per_epoch)
  estimated_epochs = int(total_steps/steps_per_epoch)
  lr_warmup_steps = int(total_steps*lr_warmup_ratio)

  for folder, (img, rep) in images_repeats.items():
    print("📁"+folder.replace("/content/drive/", "") + (" (Regularization)" if folder in reg else ""))
    print(f"📈 Found {img} images with {rep} repeats, equaling {img*rep} steps.")
  print(f"📉 Divide {pre_steps_per_epoch} steps by {train_batch_size} batch size to get {steps_per_epoch} steps per epoch.")
  if max_train_epochs:
    print(f"🔮 There will be {max_train_epochs} epochs, for around {total_steps} total training steps.")
  else:
    print(f"🔮 There will be {total_steps} steps, divided into {estimated_epochs} epochs and then some.")

  if total_steps > 20000:
    print("💥 Error: Your total steps are too high. You probably made a mistake. Aborting...")
    return

  if adjust_tags:
    print(f"\n📎 Weighted tags: {'ON' if weighted_captions else 'OFF'}")
    if weighted_captions:
      print(f"📎 Will use {keep_tokens_weight} weight on {keep_tokens} activation tag(s)")
    print("📎 Adjusting tags...")
    adjust_weighted_tags(folders, keep_tokens, keep_tokens_weight, weighted_captions)

  return True

def adjust_weighted_tags(folders, keep_tokens: int, keep_tokens_weight: float, weighted_captions: bool):
  weighted_tag = re.compile(r"\((.+?):[.\d]+\)(,|$)")
  for folder in folders:
    for txt in [f for f in os.listdir(folder) if f.lower().endswith(".txt")]:
      with open(os.path.join(folder, txt), 'r') as f:
        content = f.read()
      # reset previous changes
      content = content.replace('\\', '')
      content = weighted_tag.sub(r'\1\2', content)
      if weighted_captions:
        # re-apply changes
        content = content.replace(r'(', r'\(').replace(r')', r'\)').replace(r':', r'\:')
        if keep_tokens_weight > 1:
          tags = [s.strip() for s in content.split(",")]
          for i in range(min(keep_tokens, len(tags))):
            tags[i] = f'({tags[i]}:{keep_tokens_weight})'
          content = ", ".join(tags)
      with open(os.path.join(folder, txt), 'w') as f:
        f.write(content)

def create_config():
  global dataset_config_file, config_file, model_file

  if override_config_file:
    config_file = override_config_file
    print(f"\n⭕ Using custom config file {config_file}")
  else:
    config_dict = {
      "additional_network_arguments": {
        "unet_lr": unet_lr,
        "text_encoder_lr": text_encoder_lr,
        "network_dim": network_dim,
        "network_alpha": network_alpha,
        "network_module": network_module,
        "network_args": network_args,
        "network_train_unet_only": True if text_encoder_lr == 0 else None,
        "network_weights": continue_from_lora if continue_from_lora else None
      },
      "optimizer_arguments": {
        "learning_rate": unet_lr,
        "lr_scheduler": lr_scheduler,
        "lr_scheduler_num_cycles": lr_scheduler_num_cycles if lr_scheduler == "cosine_with_restarts" else None,
        "lr_scheduler_power": lr_scheduler_power if lr_scheduler == "polynomial" else None,
        "lr_warmup_steps": lr_warmup_steps if lr_scheduler != "constant" else None,
        "optimizer_type": optimizer,
        "optimizer_args": optimizer_args if optimizer_args else None,
      },
      "training_arguments": {
        "max_train_steps": max_train_steps,
        "max_train_epochs": max_train_epochs,
        "save_every_n_epochs": save_every_n_epochs,
        "save_last_n_epochs": keep_only_last_n_epochs,
        "train_batch_size": train_batch_size,
        "noise_offset": None,
        "clip_skip": 2,
        "min_snr_gamma": min_snr_gamma_value,
        "weighted_captions": weighted_captions,
        "seed": 42,
        "max_token_length": 225,
        "xformers": XFORMERS,
        "lowram": COLAB,
        "max_data_loader_n_workers": 8,
        "persistent_data_loader_workers": True,
        "save_precision": "fp16",
        "mixed_precision": "fp16",
        "output_dir": output_folder,
        "logging_dir": log_folder,
        "output_name": project_name,
        "log_prefix": project_name,
      },
      "model_arguments": {
        "pretrained_model_name_or_path": model_file,
        "v2": custom_model_is_based_on_sd2,
        "v_parameterization": True if custom_model_is_based_on_sd2 else None,
      },
      "saving_arguments": {
        "save_model_as": "safetensors",
      },
      "dreambooth_arguments": {
        "prior_loss_weight": 1.0,
      },
      "dataset_arguments": {
        "cache_latents": True,
      },
    }

    for key in config_dict:
      if isinstance(config_dict[key], dict):
        config_dict[key] = {k: v for k, v in config_dict[key].items() if v is not None}

    with open(config_file, "w") as f:
      f.write(toml.dumps(config_dict))
    print(f"\n📄 Config saved to {config_file}")

  if override_dataset_config_file:
    dataset_config_file = override_dataset_config_file
    print(f"⭕ Using custom dataset config file {dataset_config_file}")
  else:
    dataset_config_dict = {
      "general": {
        "resolution": resolution,
        "shuffle_caption": shuffle_caption,
        "keep_tokens": keep_tokens,
        "flip_aug": flip_aug,
        "caption_extension": caption_extension,
        "enable_bucket": True,
        "bucket_reso_steps": 64,
        "bucket_no_upscale": False,
        "min_bucket_reso": 320 if resolution > 640 else 256,
        "max_bucket_reso": 1280 if resolution > 640 else 1024,
      },
      "datasets": toml.loads(custom_dataset)["datasets"] if custom_dataset else [
        {
          "subsets": [
            {
              "num_repeats": num_repeats,
              "image_dir": images_folder,
              "class_tokens": None if caption_extension else project_name
            }
          ]
        }
      ]
    }

    for key in dataset_config_dict:
      if isinstance(dataset_config_dict[key], dict):
        dataset_config_dict[key] = {k: v for k, v in dataset_config_dict[key].items() if v is not None}

    with open(dataset_config_file, "w") as f:
      f.write(toml.dumps(dataset_config_dict))
    print(f"📄 Dataset config saved to {dataset_config_file}")

def download_model():
  global old_model_url, model_url, model_file
  real_model_url = model_url.strip()

  if real_model_url.lower().endswith((".ckpt", ".safetensors")):
    model_file = f"/content{real_model_url[real_model_url.rfind('/'):]}"
  else:
    model_file = "/content/downloaded_model.safetensors"
    if os.path.exists(model_file):
      !rm "{model_file}"

  if m := re.search(r"(?:https?://)?(?:www\.)?huggingface\.co/[^/]+/[^/]+/blob", model_url):
    real_model_url = real_model_url.replace("blob", "resolve")
  elif m := re.search(r"(?:https?://)?(?:www\\.)?civitai\.com/models/([0-9]+)(/[A-Za-z0-9-_]+)?", model_url):
    if m.group(2):
      model_file = f"/content{m.group(2)}.safetensors"
    if m := re.search(r"modelVersionId=([0-9]+)", model_url):
      real_model_url = f"https://civitai.com/api/download/models/{m.group(1)}"
    else:
      raise ValueError("optional_custom_training_model_url contains a civitai link, but the link doesn't include a modelVersionId. You can also right click the download button to copy the direct download link.")

  !aria2c "{real_model_url}" --console-log-level=warn -c -s 16 -x 16 -k 10M -d / -o "{model_file}"

  if model_file.lower().endswith(".safetensors"):
    from safetensors.torch import load_file as load_safetensors
    try:
      test = load_safetensors(model_file)
      del test
    except Exception as e:
      #if "HeaderTooLarge" in str(e):
      new_model_file = os.path.splitext(model_file)[0]+".ckpt"
      !mv "{model_file}" "{new_model_file}"
      model_file = new_model_file
      print(f"Renamed model to {os.path.splitext(model_file)[0]}.ckpt")

  if model_file.lower().endswith(".ckpt"):
    from torch import load as load_ckpt
    try:
      test = load_ckpt(model_file)
      del test
    except Exception as e:
      return False

  return True

def main():
  global dependencies_installed

  if COLAB and not os.path.exists('/content/drive'):
    from google.colab import drive
    print("📂 Connecting to Google Drive...")
    drive.mount('/content/drive')

  for dir in (main_dir, deps_dir, repo_dir, log_folder, images_folder, output_folder, config_folder):
    os.makedirs(dir, exist_ok=True)

  if not validate_dataset():
    return

  if not dependencies_installed:
    print("\n🏭 Installing dependencies...\n")
    t0 = time()
    install_dependencies()
    t1 = time()
    dependencies_installed = True
    print(f"\n✅ Installation finished in {int(t1-t0)} seconds.")
  else:
    print("\n✅ Dependencies already installed.")

  if old_model_url != model_url or not model_file or not os.path.exists(model_file):
    print("\n🔄 Downloading model...")
    if not download_model():
      print("\n💥 Error: The model you selected is invalid or corrupted, or couldn't be downloaded. You can use a civitai or huggingface link, or any direct download link.")
      return
    print()
  else:
    print("\n🔄 Model already downloaded.\n")

  create_config()

  print("\n⭐ Starting trainer...\n")
  os.chdir(repo_dir)

  !accelerate launch --config_file={accelerate_config_file} --num_cpu_threads_per_process=1 train_network.py --dataset_config={dataset_config_file} --config_file={config_file}

  if not get_ipython().__dict__['user_ns']['_exit_code']:
    display(Markdown("[Download your Lora from Google Drive](https://drive.google.com/drive/my-drive)\n"
                     "There will be several files, you should try the latest version (the file with the largest number next to it)"))

start_time_make_lora = time.perf_counter()
main()

end_time_make_lora = time.perf_counter()
time_total_make_lora = end_time_make_lora - start_time_make_lora
time_total_timedelta = timedelta(seconds=time_total_make_lora)

# Extract hours, minutes, and seconds from the timedelta
hours, remainder = divmod(time_total_timedelta.seconds, 3600)
minutes, seconds = divmod(remainder, 60)

# Format the time
creation_time = f"{hours:02d}:{minutes:02d}:{seconds:02d}"

display(Markdown(f"### ✅ [{project_name}] is Done! Lora Creation Process took: {creation_time}"))

#Create Log file
def create_log_text(project_name, log_file_path, image_count, model_name, flip_aug, num_repeats, unit, num_epochs_or_steps, batch_size, total_steps, resolution, network_dim, creation_time):
    log_data = [
        f"Trained on: {image_count} images",
        f"Training Model: {model_name}",
        f"flip_aug: {flip_aug}",
        f"Num of Repeats: {num_repeats}",
        f"Unit is Epochs or Steps: {unit}",
        f"Number of Epochs or Steps: {num_epochs_or_steps}",
        f"Training Batch Size: {batch_size}",
        f"Total Steps: {total_steps}",
        f"Resolution: {resolution}",
        f"Network Dim: {network_dim}",
        f"Lora Creation Process took: {creation_time}",
    ]

    # Create a unique log filename based on the current timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_filename = f"{project_name}_{timestamp}.txt"
    log_path_and_name = os.path.join(log_file_path, log_filename)
    # Write the data to the text file
    with open(log_path_and_name, "w") as text_file:
        text_file.write("\n".join(log_data))

    return log_path_and_name


image_count = count_images_in_folder(images_folder)
unit = preferred_unit
num_epochs_or_steps = how_many


log_filename = create_log_text(project_name, log_folder, image_count, training_modelLogName, flip_aug, num_repeats, unit, num_epochs_or_steps, train_batch_size, total_steps, resolution, dim_to_use_text, creation_time)
print(f"Log file created: {log_filename}")




# Utilities

In [ ]:
#@title Run this Cell to Disconnect from the Runtime
#@markdown This is useful if you have a long running process and you want to automatically disconnect once its done. It helps you not waste your free GPU time limit.
from google.colab import runtime
runtime.unassign()

In [ ]:
#@title Reconnect to Google Drive
#@markdown Use this if Google Colab is not seeing recently changed items in Google Drive.

# Unmount Google Drive
drive.flush_and_unmount()

# Mount Google Drive
print("📂 Connecting to Google Drive...")
drive.mount('/content/drive')


In [ ]:
#@title Duplicate a folder
#@markdown Use this if you want to make multiple projects with the same training data. This is hardcoded to get datasets from Google Drive
main_dir      = os.path.join("/content", "drive/MyDrive/lora_training")

local_var_working_dir = os.path.join(main_dir, "datasets")
folder_to_duplicate = "" #@param {type:"string"}
duplicate_folder_name = "" #@param {type:"string"}

%ls
%cp -av {local_var_working_dir}/{folder_to_duplicate} {local_var_working_dir}/{duplicate_folder_name}


In [ ]:
#@title Check if the folder exists
import os
#@markdown Use this to make sure your dataset is where it should be.
project_name = "" #@param {type:"string"}
project_name = project_name.strip()
root_dir = "/content"
main_dir      = os.path.join(root_dir, "drive/MyDrive/lora_training")
images_folder = os.path.join(main_dir, "datasets", project_name)
if(not os.path.exists(images_folder)):
  print("Error Folder does not exist")
else:
  print(f"Number of images in folder is {colabUtilities.countNumberOfImagesInFolder(images_folder)}")

